# Charts
_Roboquant_ notebooks come out of the box with support for several types of charts. These charts will help to better understand: 

1. what is going on during a run and evaluate the performance of the underlying strategy and policy. 
2. visualize the data feed you are using

<div class="alert alert-block alert-info">
    <b>Tip:</b> when using Jupyter Lab, roboquant will by default adapt to the configured theme. So if you select the dark theme in the settings menu of Jupyter Lab, the charts rendered by roboquant from then on will use an appropriate color scheme and everything will look great.
</div>

In [1]:
%use roboquant(3.0.0-SNAPSHOT)
Welcome()

roboquant 3.0.0-SNAPSHOT 
 build: 2025-05-06T09:22:02Z 
 home: /Users/peter/.roboquant 
 os: Mac OS X 15.4.1 
 jvm: OpenJDK 64-Bit Server VM 17.0.15 
 memory: 3256 MB 
 cpu cores: 8

## Price Feeds

A feed represents a stream of events you can use in your strategies. There are several type of feeds supported, like the one for Avro files used here. 

The file we are using in this notebook contains few years of S&P 500 daily prices (OHLCV). The Avro file format allows to store, access and compress data efficiently and works great for historic prices used in back-testing. 

This feed implementation also has low memory consumption since it only loads data when required. If you want to find out more about Avro files, check out their [documentation pages](https://avro.apache.org/docs/current/index.html).

In [2]:
val feed = AvroFeed.sp25()

## Performance Chart
This chart enables you to see which assets got most traded and how they performed:

1. The volume decides how big the asset is plotted. The volume = (price of the asset denoted in the base currency) x (the number unit traded).
2. The return (in %) decides the color that is used. 

You can use the slider at the top to filter an a particular range of interest, for example only the big losers.

In [3]:
val timeframe = Timeframe.fromYears(2022, 2023)
PerformanceChart(feed, timeframe = timeframe)

<!-- roboquant PerformanceChart -->

## Price Chart
Roboquant charts aren't designed to be a tool where you manually draw support lines on price charts to make trading decisions. 

But that doesn't mean price charts cannot come in handy. For example to validate if the data in a feed is not corrupted or has some other strange anomalies. 

There are two type of price charts:

1. PriceChart, this one support any type of pricing info

2. PriecBarChart, that only support price-bars (candlestick) data

A price chart will always plot the prices for a single asset. If we don't supply a timeframe, so all prices in the feed will be displayed. In the top right part of the plotted graph there is a toolbox that you can use to zoom into a particular area of interest.

In [4]:
val apple = Stock("AAPL")
val tesla = Stock("TSLA")
val microsoft = Stock("MSFT")

PriceChart(feed, apple, timeframe = timeframe)

<!-- roboquant PriceChart -->

In [5]:
val ind = TaLibIndicator.bbands()
val chart = PriceChart(feed, apple, priceType = "CLOSE", indicators = arrayOf(ind))
chart.height = 900
chart

<!-- roboquant PriceChart -->

In [6]:
val ema26 = TaLibIndicator.ema(26)
val ema12 = TaLibIndicator.ema(12)
PriceBarChart(feed, apple, indicators = arrayOf(ema12, ema26))

<!-- roboquant PriceBarChart -->

### Correlation Chart
This chart shows the correlation matrix of the prices of a number of assets within a feed. In this example we select some well popular stocks from different sectors and see how these are correlated.

In [7]:
CorrelationChart(feed, listOf(apple, tesla, microsoft))

<!-- roboquant CorrelationChart -->

### Indicators
To get further insights into the price behavior of assets, you can use indicators. You can define your own, use one of the prebuilt ones like the `rsi` indicator in the example below.

In [8]:
val rsi = TaLibIndicator.rsi(10)
val data = feed.apply(rsi, apple, tesla)
TimeSeriesChart(data)

<!-- roboquant TimeSeriesChart -->

### Strategy & Signals

Before running a back-test, you might want to validate the type of signals that your strategy is producing. You can do so, using the SignalChart. This chart plots the signals and the rating (value from -2 to 2) over time.
For this notebook we'll use the EMA crossover strategy with its default settings

In [9]:
val strategy = EMACrossover()
SignalChart(feed, strategy)

<!-- roboquant SignalChart -->

### Account
We run a simple strategy over the feed we just created to get some results to use for the charts. We'll also capture the AccountMetric metrics. After the run has finished, the account contains the latest state. There are several charts that provide some insights into this. 

In [10]:
val journal = MemoryJournal(AccountMetric())
val account = run(feed, strategy, journal=journal)
account

last update  : 2024-12-31T21:00:00Z
cash         : USD 934,623.07
buying Power : USD 934,623.07
equity       : USD 1,125,093.65
positions    : 0@MA, 40@V, 53@AMZN, 0@MSFT, 16@NFLX, 94@XOM, 23@BRK.B, 0@HD, 67@JNJ, 62@ABBV, 65@GOOG, 19@UNH, 66@GOOGL, 14@LLY, 0@META, 45@TSLA, 0@CVX, 0@WMT, 0@JPM, 60@AVGO, 0@PG, 0@MRK, 48@AAPL, 77@NVDA, 12@COST
open orders  : -16@NFLX, -53@AMZN, -77@NVDA

## Asset Allocation Charts
Lets see what assets are in our portfolio at the end of the run. We can plot only the assets (pie chart) or plot the assets grouped per asset-class (sunburst chart). Optionally we can also say if the cash positions in the account should be included in the chart.

In [11]:
AllocationChart(account)

<!-- roboquant AllocationChart -->

In [12]:
// You can of course also just print a summary
account.positions.values

[Position(size=0, avgPrice=520.86, mktPrice=526.57, lastUpdate=2024-12-31T21:00:00Z), Position(size=40, avgPrice=256.822, mktPrice=315.509, lastUpdate=2024-12-31T21:00:00Z), Position(size=53, avgPrice=201.45, mktPrice=219.39, lastUpdate=2024-12-31T21:00:00Z), Position(size=0, avgPrice=439.33, mktPrice=421.5, lastUpdate=2024-12-31T21:00:00Z), Position(size=16, avgPrice=617.52, mktPrice=891.32, lastUpdate=2024-12-31T21:00:00Z), Position(size=94, avgPrice=120.32, mktPrice=107.57, lastUpdate=2024-12-31T21:00:00Z), Position(size=23, avgPrice=458.49, mktPrice=453.28, lastUpdate=2024-12-31T21:00:00Z), Position(size=0, avgPrice=416.98, mktPrice=388.99, lastUpdate=2024-12-31T21:00:00Z), Position(size=67, avgPrice=146.16, mktPrice=144.62, lastUpdate=2024-12-31T21:00:00Z), Position(size=62, avgPrice=176.19, mktPrice=177.7, lastUpdate=2024-12-31T21:00:00Z), Position(size=65, avgPrice=166.99, mktPrice=190.44, lastUpdate=2024-12-31T21:00:00Z), Position(size=19, avgPrice=489.34, mktPrice=505.86, last

In [13]:
PriceBarChart(feed, apple)

<!-- roboquant PriceBarChart -->

# Metrics

Besides the charts for account related data, we can also plot the metrics that where captured during the run. Where an Account contains only the end state after the run has finished, metrics are captured during each step of the run. This allows for charts that provide more insights into different periods of the run. We used the AccountMetric metric that keeps track of how our account performs during the back-test. We use the account value (= cash + positions), for this sample

In [14]:
// What metrics are available
journal.getMetricNames()

[account.orders, account.positions, account.cash, account.buyingpower, account.equity, account.mdd]

## Metric Chart

In [15]:
val data = journal.getMetric("account.equity")
TimeSeriesChart(data)

<!-- roboquant TimeSeriesChart -->

## Metric Box Chart
If you want to inspect how volatile the performance of a strategy is, the metric box chart comes in handy. It provides insights into how a metric is distributed over a period of time. If provides the following info per box:

- The minimum
- low percentile (default is 25.0)
- mid percentile (default is 50.0, aka median)
- high percentile (default is 75.0)
- the maximum 

So in this case it shows these statistics of the **daily equity returns** based on the monthly aggregate of daily values.

In [16]:
BoxChart(data.returns())

<!-- roboquant BoxChart -->

In [17]:
BoxChart(data.returns(), period=ChronoUnit.YEARS, lowPercentile = 10.0, highPercentile = 90.0)

<!-- roboquant BoxChart -->

## Metric Histogram Chart
If we want to see the distribution of the metric values, we can use the MetricHistogram chart. Besides the metric data that needs to be plot, you can also specify the number of bins. On the x-axis the upperbound of each bin is displayed and on the y-axis how many observations fit in that bin. 

In [18]:
HistogramChart(data.diff(), 15)

<!-- roboquant HistogramChart -->

## Calender Chart
When evaluating the performance of a strategy, it is sometimes required to find out when the strategy was performing good and bad. A calendar charts allows you to plot a metric value per day, so it is easy to find out which days to further investigate. The slider at top of the chart allows for filtering only those days that are of interest based on their value.

In the example below we plot the daily account change and loss to see when the strategy performed good (green) and when bad (red). It also demonstrates how to display metrics only for a limited timeframe.

In [19]:
CalendarChart(data.diff())

<!-- roboquant CalendarChart -->

# Multi-run tests
A good way to back-test your strategy is to run it multiple times over different time-frames. This section shows how charts that come in handy to better understand the results of those type of back-tests. 

## Walk-forward
A commonly used approach in back-testing is the so called walk-forward. The historic data is split in a number of equal periods and then the strategy is run for each period separately. This shows how the strategy would have performed during these different periods and possible different regimes.

In the following code we split the data in periods of 1 year and for each period run it. We then plot again the account value as observed during each run.

In [20]:
val mrj = MultiRunJournal {
    MemoryJournal(AccountMetric())
}

feed.timeframe.split(12.months).forEach {
    run(feed, EMACrossover(), mrj.getJournal(), timeframe = it)
}

val data = mrj.getMetric("account.equity")
TimeSeriesChart(data)

<!-- roboquant TimeSeriesChart -->

## Monte Carlo Simulation
Another great way to back-test your strategy is to run it for a fixed duration at many random starting points in time. This gives you a good insight what could be a good and bad outcome of your strategy. The following piece of code runs the strategy 25 times, each time for the duration of 6 months. Feel free to change the number of runs from 25 to a higher value like 50 or 100. It is kept small so that also people with older or less powerful computers don't have to wait too long.

In this case we plot the *account value* (= cash + positions) again and you can see there are times we do well but there are also periods when we loose money.

In [21]:
val mrj = MultiRunJournal {
    MemoryJournal(AccountMetric())
}

feed.timeframe.sample(12.months, 50).forEach {
     run(feed, EMACrossover(), mrj.getJournal(), timeframe = it)
}

val data = mrj.getMetric("account.equity")
TimeSeriesChart(data, useTime=false)

<!-- roboquant TimeSeriesChart -->

If you want to see the same data, but this time plotted in time, you can use the same TimeSeriesChart class and provide the *useTime=true* parameter. This shows in which periods the strategy is performing well and in which periods it is struggling. It is similar to the walk-forward approach, but provides at each moment in time multiple samples.

In [22]:
TimeSeriesChart(data, useTime=true)

<!-- roboquant TimeSeriesChart -->

# Displaying multiple charts
By using the *chart.render()* method you can display multiple charts in the output section of a single cell. 

In [27]:
val assets = listOf(Stock("AAPL"), Stock("JPM"), Stock("GOOGL"), Stock("MSFT"))
for (asset in assets.take(2)) PriceBarChart(feed, asset).render()

<!-- roboquant PriceBarChart -->

<!-- roboquant PriceBarChart -->

Another example, lets see how the correlation between assets differs over timeframes of maximum 1 year.

In [28]:
feed.timeframe.split(1.years).forEach {
    val chart = CorrelationChart(feed, assets, it)
    chart.title = "correlations $it"
    chart.render()
}

<!-- roboquant CorrelationChart -->

<!-- roboquant CorrelationChart -->

<!-- roboquant CorrelationChart -->

<!-- roboquant CorrelationChart -->